In [19]:
import pandas as pd
import nltk
from nltk.metrics.agreement import AnnotationTask

joanna = pd.read_csv('to_annotate_joanna_finished.csv')

joanna.head()

,Unnamed: 0,created_at,tweet_id,text,lang,user_info,company,rating
0,5528,Mar 03 2020,1.234916e+18,#Search #Google propose trois moteurs de reche...,fr,"{'id': 16578116, 'id_str': '16578116', 'name':...",Google,Neutral
1,77,Feb 27 2020,1.233167e+18,@AilesCyclables @RasLeScoot On parle de Retro ...,fr,"{'id': 1076211314563997702, 'id_str': '1076211...",Facebook,Irrelevant
2,2763,Mar 04 2020,1.235346e+18,@akasugarcane Ils étaient grave bien en plus !...,fr,"{'id': 1184061352475856899, 'id_str': '1184061...",Facebook,Irrelevant
3,3782,Feb 28 2020,1.233388e+18,L Escoleta De Maru SL en #Google https://t.co/...,fr,"{'id': 3292548718, 'id_str': '3292548718', 'na...",Google,Irrelevant
4,2100,Mar 02 2020,1.234614e+18,@AmazonHelp Bon @amazonca @AmazonHelp visiblem...,fr,"{'id': 1166021798, 'id_str': '1166021798', 'na...",Amazon,Very Negative


In [4]:
molly = pd.read_csv('to_annotate_molly_finished.csv')

molly.head()

,Unnamed: 0,created_at,tweet_id,text,lang,user_info,company,rating
0,5528,Mar 03 2020,1.234916e+18,#Search #Google propose trois moteurs de reche...,fr,"{'id': 16578116, 'id_str': '16578116', 'name':...",Google,Neutral
1,77,Feb 27 2020,1.233167e+18,@AilesCyclables @RasLeScoot On parle de Retro ...,fr,"{'id': 1076211314563997702, 'id_str': '1076211...",Facebook,Irrelevant
2,2763,Mar 04 2020,1.235346e+18,@akasugarcane Ils étaient grave bien en plus !...,fr,"{'id': 1184061352475856899, 'id_str': '1184061...",Facebook,Irrelevant
3,3782,Feb 28 2020,1.233388e+18,L Escoleta De Maru SL en #Google https://t.co/...,fr,"{'id': 3292548718, 'id_str': '3292548718', 'na...",Google,Irrelevant
4,2100,Mar 02 2020,1.234614e+18,@AmazonHelp Bon @amazonca @AmazonHelp visiblem...,fr,"{'id': 1166021798, 'id_str': '1166021798', 'na...",Amazon,Negative


In [31]:
molly_ratings = list(molly.iloc[:, 7])
joanna_ratings = list(joanna.iloc[:, 7])
sents = list(molly.iloc[:, 3])
#sents2 = ([word for sent in sents for word in sent.split(" ")])

print(molly_ratings[:5])
print(len(molly_ratings))
print(joanna_ratings[:5])
print(len(joanna_ratings))
print(sents[:3])
print(len(sents))
print(sents2[:3])

['Neutral', 'Irrelevant', 'Irrelevant', 'Irrelevant', 'Negative']
760
['Neutral', 'Irrelevant', 'Irrelevant', 'Irrelevant', 'Very Negative']
760
['#Search #Google propose trois moteurs de recherche alternatifs sur Android\nhttps://t.co/vqN0T3SWvb\n\nvia @01net', '@AilesCyclables @RasLeScoot On parle de Retro Tours Paris? Qui a une page Facebook?', '@akasugarcane Ils étaient grave bien en plus ! Jusqu’à j’avais fais une page Facebook']
760
['#Search', '#Google', 'propose']


In [16]:
annots = {"molly": molly_ratings, "joanna": joanna_ratings}

In [48]:
def polarity_distance2(tag1,tag2):
    annot_dict = {"Very Positive": 5, "Positive": 4, "Neutral": 3, "Negative": 2, "Very Negative": 1}
    if tag1 == tag2:
        return 0
    elif tag1 == "Irrelevant" or tag2 == "Irrelevant":
        return 5
    else:
        return abs(annot_dict[tag1] - annot_dict[tag2])
    
def convert_to_triples(sentence,annotations_by_annotator):
    triple_list = []
    for annotator, annotations in annotations_by_annotator.items():
        for i in range(len(annotations)):
            triple_list.append((annotator,sentence[i],annotations[i]))
    return triple_list

In [56]:
triples = convert_to_triples(sents, annots)
annotation_task = AnnotationTask(triples, distance=polarity_distance2)
annotation_task.alpha()

0.8310376034842383

In [50]:
triples

[('molly',
  '#Search #Google propose trois moteurs de recherche alternatifs sur Android\nhttps://t.co/vqN0T3SWvb\n\nvia @01net',
  'Neutral'),
 ('molly',
  '@AilesCyclables @RasLeScoot On parle de Retro Tours Paris? Qui a une page Facebook?',
  'Irrelevant'),
 ('molly',
  '@akasugarcane Ils étaient grave bien en plus ! Jusqu’à j’avais fais une page Facebook',
  'Irrelevant'),
 ('molly',
  'L Escoleta De Maru SL en #Google https://t.co/ZzxX651GNY',
  'Irrelevant'),
 ('molly',
  '@AmazonHelp Bon @amazonca @AmazonHelp visiblement votre livreur est incapable de livrer cette fois le motif invoqué… https://t.co/ujvAbZRUH4',
  'Negative'),
 ('molly',
  "Jvous aurait bien passée mon Amazon prime pour regarder love island mais ya ma CB c'est mort",
  'Negative'),
 ('molly',
  '"Microsoft Announcements on Azure Artificial Intelligence" #MachineLearning https://t.co/t91A3GpLUh',
  'Neutral'),
 ('molly',
  '@JoestarNoisette @Moowmoowgli @zoubzoub_ @ZeeDrDaft @unaberberian @NazarethGatsu T’as pas 

In [53]:
trip_df = pd.DataFrame(triples)

In [54]:
trip_df.to_csv('triples_df.csv')